In [1]:
import torchvision.models as models
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
import foolbox.attacks as fa

from models.VGG16 import *

from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms

use_cuda=True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

# ======================================== prepare the dataset ==========================================================================================
mean_cifar10 = [0.485, 0.456, 0.406]   
std_cifar10 = [0.229, 0.224, 0.225]
batch_size = 16

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_cifar10,std_cifar10),
])

testset = torchvision.datasets.CIFAR10(root='../FlexibleCNNs/data', train=False, download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)

In [2]:
if __name__ == "__main__":
    # instantiate a model
    model = VGG16().to(device)
        
    state = torch.load('./models/model_150_89.pth')
    model.load_state_dict(state['model'])    
    model.eval()
    
    preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
    fmodel = PyTorchModel(model, bounds=(-1, 1), preprocessing=preprocessing)
    
    images, labels = ep.astensors(*samples(fmodel, dataset="cifar10", batchsize=1000))
    print(accuracy(fmodel, images, labels))
        
    # apply the attack
    #attack = fa.boundary_attack.BoundaryAttack(
    #steps=25000,
    #)
    attack = fa.LinfPGD()
    epsilons = [0.0]
    for data in testloader:
        images, labels = data
        advs, _, success = attack(fmodel, images, labels, epsilons=epsilons)

    # calculate and report the robust accuracy
    robust_accuracy = 1 - success.float32().mean(axis=-1)
    for eps, acc in zip(epsilons, robust_accuracy):
        print(eps, acc.item())

    # we can also manually check this
    for eps, advs_ in zip(epsilons, advs):
        print(eps, accuracy(fmodel, advs_, labels))
        # but then we also need to look at the perturbation sizes
        # and check if they are smaller than eps
        print((advs_ - images).norms.linf(axis=(1, 2, 3)).numpy())

/home/mlavrov/anaconda3/envs/meng/lib/python3.7/site-packages/foolbox/utils.py:81: UserWarning: samples() has only 20 samples and repeats itself if batchsize > 20
  "samples() has only 20 samples and repeats itself if batchsize > 20"


0.9000000357627869


RuntimeError: expected device cpu but got device cuda:0

In [3]:
print(images.shape)

torch.Size([16, 3, 32, 32])


epsilon = [0.0]

if __name__ == "__main__":
    # instantiate a model
    model = VGG16().to(device)
        
    state = torch.load('./models/model_150_89.pth')
    model.load_state_dict(state['model'])    
    model.eval()
    
    preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
    fmodel = PyTorchModel(model, bounds=(-2.117, 2.64), preprocessing=preprocessing)
        
    # apply the attack
    attack = fa.boundary_attack.BoundaryAttack(
    steps=25000,
    )
    
    #evaluate model
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # Call Boundary Attack
            perturbed_data, _, success  = attack(fmodel, images, labels, epsilons=epsilon)
            # Re-classify the perturbed image
            outputs = model(perturbed_data)            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy on the test dataset is: %f %%' % (
        100 * correct / total))



    # get data and test the model
    # wrapping the tensors with ep.astensors is optional, but it allows
    # us to work with EagerPy tensors in the following
    #images, labels = ep.astensors(*samples(fmodel, dataset="cifar10", batchsize=16, shape=(32, 32), bounds=(-1, 1)))
    #print(accuracy(fmodel, images, labels))


    
    
    """
    epsilons = [0.0, 0.001]
    xp_, _, success = attack(fmodel, images, labels, epsilons=epsilons)
    print(
        "Boundary attack success within specified parameters",
        success.numpy().astype(np.float32).mean() * 100,
        " %",
    )
    
    robust_accuracy = 1 - success.float32().mean(axis=-1)
    print(robust_accuracy)
   
    
    epsilons = [0.0, 0.03, 0.1, 0.3]
    advs, _, success = attack(fmodel, images, labels, epsilons=epsilons)

    # calculate and report the robust accuracy
    robust_accuracy = 1 - success.float32().mean(axis=-1)
    for eps, acc in zip(epsilons, robust_accuracy):
        print(eps, acc.item())
     """